In [1]:
# HIDDEN - sets path

import sys
sys.path.insert(1, '../..')

# Writing detected anomalies to TrendMiner
In this use cases, anomalies detected by an external service need to be written to TrendMiner as context items, where they can be consulted and queried by process experts. Additionally, we want to provide a direct link to the anomaly as a TrendHub view in the external service.

## The anomalies
The anomalies are identified by an external service, and are in the following format:

In [2]:
anomalies = [
    {
        "anomaly type": "ANOMALY",  # Context type key
        "asset": "AF-example/Europe/Belgium/Hasselt", # Path to asset
        "anomalous tags": ["[CS]BA:LEVEL.1", "[CS]BA:CONC.1"], # Anomalous data (visible in view)
        "reference tags": ["[CS]BA:TEMP.1", "[CS]BA:PHASE.1"], # Additional info for user (hidden in view)
        "start": "2023-01-23 15:17:23",
        "end": "2023-01-23 16:01:04",
        "description": "",
        "keywords": ["anomaly", "l1"],
        "fields": {
            "impact": "High"
        }
    }
]

## Creating the client
Setting the timezone is important as event timestamps will be assumed to be in this timezone. We need to browse the asset framework, so we need to add local user authentication (username + password) as a client user cannot be given asset permissions at this time. This local user should of course have permissions on the assets we want to access in this workflow.

In [3]:
import keyring
from trendminer_interface import TrendMinerClient

url = "https://cs.trendminer.net"
client_id = "tmInterfaceClient"
username = "wdaniels"

client = TrendMinerClient(
    url = url,
    client_id = client_id,
    client_secret=keyring.get_password(url, client_id),
    username=username,
    password=keyring.get_password(url, username),
    tz="Europe/Brussels"
)

## Anomalies to context items
The anomalies can be converted into TrendMiner context items

In [4]:
items = []

for anomaly in anomalies:
    context_type = client.context.type.from_key(anomaly["anomaly type"])
    asset = client.asset.from_path(anomaly["asset"])
    context_item = client.context.item(
        context_type=context_type,
        components=[asset],
        fields=anomaly["fields"],
        events=[anomaly["start"], anomaly["end"]],
        description=anomaly["description"],
        keywords=anomaly["keywords"],
    )
    items.append(context_item)

items

[<< ContextItem | ANOMALY | 0:43:41 >>]

These items can be written to the appliance

In [5]:
for item in items:
    item.post()

## Links to context items
We now generate customized TrendHub views to which we generate a unique url. These urls allow the user of the anomaly detection tool to immediately open the corresponding view in TrendMiner (after logging in). Note that we do not actually save the generated views to our work organizer, we simply instantiate them to generate the url.

In [6]:
for anomaly in anomalies:
    
    # core anomaly tags need to be displayed`
    anomaly_tags = client.tag.list(anomaly["anomalous tags"])
    
    # other tags need to be hidden in the view, but accessible to the user
    reference_tags = client.tag.list(anomaly["reference tags"])
    for tag in reference_tags:
        tag.visible = False
     
    # main view attributes
    tags = anomaly_tags + reference_tags       
    interval = client.interval(anomaly["start"], anomaly["end"])
    
    # chart configuration
    chart = client.trend.chart.stacked(
        locked=True,
        grid=False,
        filling=False,
        context=True,
    )
    
    # view configuration
    thv = client.trend.view(
        entries=tags, 
        layers=[interval], 
        live=False,
        context_interval="3y",
        chart=chart,
    )
    
    # Generate unique url to view
    print(thv.get_session_url())

https://cs.trendminer.net/trendhub/#/share/cd6dee795f04


In [7]:
# HIDDEN - item cleanup
for item in items:
    item.delete()